# Extract and save as JSON

In [10]:
from bs4 import BeautifulSoup
from datetime import datetime
import glob
import re
import os
import json
import locale
locale.setlocale(locale.LC_TIME, "de_ch")

'de_ch'

In [11]:
path_in = '../data/article/'
path_out = '../data_cleaned/article/'

In [12]:
def one_or_warn(selector):
    l = soup.select(selector)
    if(len(l) > 1):
        print("More than one %s at %s" % (selector, f))
    if(len(l) > 0):
        return l[0].string
    return ""

reg_abbr = re.compile('\((([^)]*))\)[^(]*$', re.IGNORECASE)

records = []
filelist = glob.glob("%s*.html" % path_in)
for f in filelist:
    #f = '../data/article/anerkennung-meines-einsatzes-fuer-menschenrechte.html'
    
    # Get ID
    id = os.path.basename(f).replace('.html', '')
    
    # Check if exists
    if os.path.isfile('%s%s.json' % (path_out, id)):
        continue
        
        
    print(f)
    
    record = {'id': id}
    
    soup = BeautifulSoup(open(f, encoding='utf-8'), "html.parser")

    # Find Tag
    record['tag'] = one_or_warn('.esf-img-tag span')

    # Find Title
    record['title'] = one_or_warn('h1')

    # Lead
    #record['lead'] = soup.select_one('.text p strong').string

    # Text
    record['text'] = soup.select_one('.text').text

    # Date
    icon = soup.select_one('img[src="https://www.kath.ch/wp-content/themes/cathkathcatt/images/icon_date.png"]')
    if(len(icon) == None):
        print("No date found at %s" % f)
    date_s = icon.parent.parent.select_one('.col-9').string.replace('\n', '').strip()
    date = date_s.split('|')[0].strip()
    time = date_s.split('|')[1].strip()

    d = datetime.strptime("%s %s" % (date, time), '%d. %B %Y %H:%M')
    record['date'] = d.strftime("%Y-%m-%d %H:%M")

    # Get author
    s = soup.select('.text>p')
    if len(s) > 2:
        s = s[1].text
        if(len(s.split(' ')) <= 4):

            # Looks like there is an author
            record['author'] = s.strip()

        else:

            # Try to guess abbrev.
            reg = reg_abbr.search(soup.select('.text>p')[-1].text)
            if reg:
                record['author'] = reg.group(1).strip()
        
    
    # Save to file
    with open('%s%s.json' % (path_out, record['id']), 'w', encoding='UTF-8') as fp:    
        json.dump(record, fp, ensure_ascii=False)
    
print("finito")


#record

../data/article\nordirischer-bischof-in-sorge-wegen-brexit.html
finito
